<a href="https://colab.research.google.com/github/La-matrice/brouhaha/blob/main/brouhaha.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Complétion de phrase en français**

In [ ]:
# Importation des fichiers sources du Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# **A - Identification des mots saisis par l'utilisateur**

In [33]:
import pandas as pd

phrase_input = input("Début de la phrase :")

print("Début de l'identification...")

df = pd.read_csv('/content/drive/MyDrive/VaCarMe/Dictionnaire.txt', sep='|', header=None, skiprows=1, encoding='utf-8')

# Initialisation de la liste de résultats
result_list = []
phrase_input = phrase_input.lower().strip()

# Boucle de recherche d'expressions
k = len(phrase_input)

while k > 0:
    found = False
    i = min(27, k)  # le nombre maximum de caractères d'une expression

    while not found:
        match_index = df[df[0] == phrase_input[:i]].index.tolist()

        # Si une correspondance est trouvée
        if match_index:
            mot = df.loc[match_index[0], 0]
            id_ = df.loc[match_index[0], 1]
            cv = df.loc[match_index[0], 2]
            freq = df.loc[match_index[0], 6]

            # Ajoute les résultats à la liste de résultats
            result_list.append([mot, id_, '', '', '', '', cv, freq])
            found = True

            # Mise à jour de phrase_input
            if ' ' in phrase_input or "'" in phrase_input:
                phrase_input = phrase_input[len(mot):].strip()
                k = len(phrase_input)
            else:
                phrase_input = ''
                k = 0
        else:
            i -= 1

        # Si aucun mot n'est trouvé
        if i == 0:
            print(f"Certains mots sont introuvables dans le dictionnaire : {phrase_input}")

# Conversion de la liste de résultats en DataFrame
result_df = pd.DataFrame(result_list, columns=['Mot', 'ID', 'ID principal probable', '% ID principal', 'ID précis probable', '% ID précis probable', 'CV', 'Fréquence'])

# Exportation du DataFrame dans un fichier texte
result_df.to_csv('result.txt', sep='|', index=False, header=False, encoding='utf-8')

for line in result_list:
    print(line)

Début de la phrase :je pense donc je
Début de l'identification...
['je', 'PRO:per:suj:e1s', '', '', '', '', 'CV', 5556]
['pense', 'VER:P0:A1:sub:pre:3s & VER:P0:A1:sub:pre:1s & VER:P0:A1:ind:pre:3s & VER:P0:A1:ind:pre:1s & VER:P0:A1:imp:2s & VER:P0:A1-N1a:sub:pre:3s & VER:P0:A1-N1a:sub:pre:1s & VER:P0:A1-N1a:ind:pre:3s & VER:P0:A1-N1a:ind:pre:1s & VER:P0:A1-N1a:imp:2s & VER:P0:N1a:sub:pre:3s & VER:P0:N1a:sub:pre:1s & VER:P0:N1a:ind:pre:3s & VER:P0:N1a:ind:pre:1s & VER:P0:N1a:imp:2s & VER:P0:T15:sub:pre:3s & VER:P0:T15:sub:pre:1s & VER:P0:T15:ind:pre:3s & VER:P0:T15:ind:pre:1s & VER:P0:T15:imp:2s & VER:P0:T14:sub:pre:3s & VER:P0:T14:sub:pre:1s & VER:P0:T14:ind:pre:3s & VER:P0:T14:ind:pre:1s & VER:P0:T14:imp:2s & VER:P0:T13-P30:sub:pre:3s & VER:P0:T13-P30:sub:pre:1s & VER:P0:T13-P30:ind:pre:3s & VER:P0:T13-P30:ind:pre:1s & VER:P0:T13-P30:imp:2s', '', '', '', '', 'CV', 217]
['donc', 'CON:coo:csq', '', '', '', '', 'CC', 738]
['je', 'PRO:per:suj:e1s', '', '', '', '', 'CV', 5556]


# **B - Evaluation des formes probables des mots identifiés**

In [34]:
#Optimiser les boucles

import pandas as pd
import io
import csv
  
#plafond = input("Profondeur d'évaluation :")
#plafond = int(plafond)
plafond = 50

print("Début de l'évaluation des formes...")

k = 1
mots_input = []
liste_resultats = []

df = pd.read_csv('result.txt', sep="|", header=None, encoding='utf-8')

# lister les mots identifiés
for index, row in df.iterrows():
    mot = row[0]
    id_ = row[1]
    cv = row[6]
    freq = row[7]

    mots_input.append([mot, id_,  None, None, None, None, cv, freq])
    k += 1

limite = k - 1

# initialiser les fichiers textes
with open('/content/drive/MyDrive/VaCarMe/Mots.txt', 'r', encoding='utf-8') as file:
    lines_mots = file.read().split("\n")
    
with io.open('/content/drive/MyDrive/VaCarMe/Dictionnaire.txt', 'r', encoding='utf-8') as fichier2:
    lines_dictionnaire = fichier2.read().split("\n")
                        
# tester chaque mot
for k in range(limite):
    j = 1
    pourcentage_forme_principale = 0
    portion = ""
    best_score = 0
    formes_probables_principales = []
    formes_probables = []

    # si le mot possède plusieurs formes, évaluer sa forme
    if "&" in mots_input[k][1]:
        print("Le mot suivant possède plusieurs formes :",mots_input[k][0])

        if limite == 1:
            print(f"Le mot {mots_input[k][0]} comporte plusieurs formes, merci de saisir plus d'un mot pour lancer la génération !")
            exit()

        # tester 4 positions différentes (de n+1, n+2 à n-1, n-2) et identifier le mot indice
        for q in range(4):
            compteur = 0
            if q < 2:
                if k + (q + 1) < limite:
                    indice = mots_input[k + (q + 1)][0]
                else:
                    continue
            else:
                if k - (q - 1) > - 1:
                    indice = mots_input[k - (q - 1)][0]
                else:
                    continue

            if indice in ['?', '!', ',', '.', '-']:
                continue

            # tester chaque ligne du fichier 'Mots'
            for line in lines_mots:
                trouve = False

                # vérifier si le mot indice existe dans la ligne
                while "|" + indice + "|" in line:

                    z = 0
                    x = 0
                    y = 1
                    mots = line.split("|")

                    # tester chaque mot trouvé dans la ligne (relever z, le nombre total de mot ; relever x, la position du mot indice dans la ligne)
                    for mot in mots:
                        z += 1
                        if mot == indice:
                            x = z
                    
                    # identifier la position en cours et relever la portion de la ligne correspondante
                    if q <= 1:
                        if x - (q + 2)> 0:
                            portion = mots[x - (q + 2)]
                        else:
                            break
                    else:
                        if x + (q + 1) <= z:
                            portion = mots[x + ((q + 1) - 3)]
                        else:   
                            break

                    # à la fin de l'opération, supprimer l'indice de la phrase et poursuivre l'opération (si la ligne possède une deuxième fois le mot indice, continuer sur la même ligne)
                    line = line.replace("|" + indice + "|", "", 1)


                    # tester chaque ligne du Dictionnaire
                    for line2 in lines_dictionnaire:
                        split_line2 = line2.split("|")
                        
                        # tester si la première partie de la ligne correspond à la portion
                        if split_line2[0] == portion:
                            forme_dictionnaire = split_line2[1]
                            
                            # Calcul du nombre de formes probables du mot trouvé
                            n = forme_dictionnaire.count("&")
                            #print("Nombre de formes :",n + 1)
                            liste_doublons = [None] * (n + 1)
                            compteur += 1
                            #print(compteur)

                            # Saisie des formes probables principales et formes probables précises ainsi que leur nombre d'occurrence
                            for m in range(n + 1):
                                doublon = False

                                # Saisie de la forme complète (forme la plus précise)
                                j = 0
                                trouve = False
                                while j < len(formes_probables) and len(formes_probables) > 0 and not trouve:
                                    if forme_dictionnaire.split(" & ")[m].strip() == formes_probables[j][0]:
                                        trouve = True
                                        formes_probables[j][0] = forme_dictionnaire.split(" & ")[m]
                                        formes_probables[j][q + 1] += 1
                                    j += 1
                                if not trouve:
                                    ligne = ["", 0, 0, 0, 0]
                                    formes_probables.append(ligne)
                                    formes_probables[j][0] = forme_dictionnaire.split(" & ")[m]
                                    formes_probables[j][q + 1] += 1

                                # Saisie de la forme principale
                                # Vérifie qu'il n'y a pas de doublon de forme principale
                                for compteur_doublon in range(n + 1):
                                    if liste_doublons[compteur_doublon - 1] == forme_dictionnaire.split(" & ")[m].strip()[:3]:
                                        doublon = True

                                if compteur_doublon == n and not doublon:
                                    liste_doublons[compteur_doublon] = forme_dictionnaire.split(" & ")[m].strip()[:3]

                                # Si pas de doublon on saisit le nombre d'occurrence
                                if not doublon:
                                    j = 0
                                    trouve = False
                                    while j < len(formes_probables_principales) and len(formes_probables_principales) > 0 and not trouve:
                                        if forme_dictionnaire.split(" & ")[m].strip()[:3] == formes_probables_principales[j][0]:
                                            trouve = True
                                            formes_probables_principales[j][0] = forme_dictionnaire.split(" & ")[m][:3]
                                            formes_probables_principales[j][q + 1] += 1

                                        j += 1
                                    if not trouve:
                                        ligne = ["", 0, 0, 0, 0]
                                        formes_probables_principales.append(ligne)
                                        formes_probables_principales[j][0] = forme_dictionnaire.split(" & ")[m][:3]
                                        formes_probables_principales[j][q + 1] += 1

                                trouve = True


                            if not trouve:
                                break

                # Si le nombre de formes probables trouvées est jugé suffisant, on arrête la recherche et on poursuit l'évaluation
                if compteur > plafond:
                    break

        # Calcul de la moyenne des positions par forme principale
        for q in range(1, 5):
            compteur = 0
            pourcentage = 0
            i = 0
            while i < len(formes_probables_principales) and formes_probables_principales[i][0] != "":
                compteur += 1
                if formes_probables_principales[i][q] > 0:
                    pourcentage += formes_probables_principales[i][q]
                i += 1

            # Saisie du pourcentage de présence d'une forme pour chaque position
            for i in range(compteur):
                if formes_probables_principales[i][q] > 0:
                    formes_probables_principales[i][q] = (formes_probables_principales[i][q] * 100) / pourcentage

        # Saisie de la moyenne
        for i in range(compteur):
            pourcentage = 0
            for q in range(1, 5):
                pourcentage += formes_probables_principales[i][q]
            formes_probables_principales[i][1] = pourcentage / 4

        # Tri des FormesProbables par ordre décroissant des moyennes
        formes_probables_principales.sort(key=lambda x: x[1], reverse=True)

        # Attribution de la forme principale la plus probable            
        compteur = mots_input[k][1].count("&")
        pourcentage = 0
        best_forme = 0
        for i in range(compteur):
            j = 0
            while j < len(formes_probables_principales):
                id_unique = mots_input[k][1].split("&")[i].strip()[:3]  # 3 pour uniquement connaitre la forme principale
                if id_unique == formes_probables_principales[j][0]:
                    if pourcentage < formes_probables_principales[j][1]:
                        best_forme = mots_input[k][1].split("&")[i].strip()
                        pourcentage = formes_probables_principales[j][1]
                        c = pourcentage
                        mots_input[k][2] = formes_probables_principales[j][0]
                        mots_input[k][3] = pourcentage
                        pourcentage_forme_principale = pourcentage
                j += 1

        # Si aucune forme similaire trouvée du mot input en cours, on attribue la forme la plus probable
        if best_forme == 0:
            mots_input[k][2] = formes_probables_principales[0][0]
            mots_input[k][3] = formes_probables_principales[0][1]

        # Calcul de la moyenne des positions par forme précise à partir de la forme principale la plus probable
        for q in range(1, 5):
            compteur = 0
            pourcentage = 0
            i = 0
            while i < len(formes_probables):
                compteur += 1
                #print(formes_probables[i][0][:3],mots_input[k][2] )
                if formes_probables[i][0][:3] == mots_input[k][2]:
                    if formes_probables[i][q] > 0:
                        pourcentage += formes_probables[i][q]
                        #print(pourcentage)
                else:
                    for p in range(1, 5):
                        formes_probables[i][p] = 0
                i += 1

            # Saisie du pourcentage de présence d'une forme pour chaque position
            for i in range(compteur):
                if formes_probables[i][q] > 0:
                    formes_probables[i][q] = (formes_probables[i][q] * 100) / pourcentage

        # Saisie de la moyenne
        for i in range(compteur):
            pourcentage = 0
            for q in range(1, 5):
                pourcentage += formes_probables[i][q]
            formes_probables[i][1] = pourcentage / 4

        # Tri des formes probables par ordre décroissant des moyennes
        formes_probables.sort(key=lambda x: x[1], reverse=True)
            
        # Attribution de la forme la plus probable
        compteur = mots_input[k][1].count("&")
        pourcentage = 0
        for i in range(compteur + 1):
            j = 0
            while j < len(formes_probables) and formes_probables[j][0] != "":
                id_unique = mots_input[k][1].split("&")[i].strip()
                if id_unique == formes_probables[j][0]:
                    if pourcentage < formes_probables[j][1]:
                        best_forme = mots_input[k][1].split("&")[i].strip()
                        best_score = formes_probables[j][1]
                        pourcentage = formes_probables[j][1]
                j += 1

        mots_input[k][4] = best_forme
        mots_input[k][5] = best_score

        print("Nouvelle forme attribuée : ", mots_input[k][4])

    # Si le mot dispose que d'une seule forme
    else:
        mots_input[k][2] = mots_input[k][1][:3]
        mots_input[k][3] = "100"
        mots_input[k][4] = mots_input[k][1]
        mots_input[k][5] = "100"
        mots_input[k][6] = cv
        mots_input[k][7] = freq

with open('result.txt', "w", encoding='utf-8', newline="") as fichier:
        writer = csv.writer(fichier, delimiter='|')
        writer.writerows(mots_input)
print("Evaluation des formes terminée")

Début de l'évaluation des formes...
Le mot suivant possède plusieurs formes : pense
Nouvelle forme attribuée :  VER:P0:T14:sub:pre:3s
Evaluation des formes terminée


#**C - Prédiction de la forme suivante probable**

In [35]:
import pandas as pd
import io
import csv
import re

print("Début de la prédiction...")

plafond_prediction = 100

formes_input = []
formes_successives = []
formes_precises_probables = []
formes_principales_probables = []

k = 0

# Lecture du fichier
df = pd.read_csv('result.txt', sep="|", header=None, encoding='utf-8')

# Itération à travers les lignes
for index, row in df.iterrows():
    mot = row[0]
    id_ = row[1]
    idp = row[2]
    idpp = row[3]
    ide = row[4]
    idep = row[5]
    cv = row[6]
    freq = row[7]

    formes_input.append([mot, id_,  idp , idpp, ide , idep , cv, freq])

    k += 1

# ouvrir le fichier source
with io.open('/content/drive/MyDrive/VaCarMe/Formes.txt', 'r', encoding='utf-8') as fichier:
    lines = fichier.readlines()

x = len(formes_input) - 1 # Nombre total de forme
p = x + 1 # Profondeur de la recherche (x+1 pour le total)

# boucler sur chaque mot jusqu'à trouver une solution viable
for p in range(1, x + 2):
    FiltreForme = 21
    trouve = False
    fin = False
    #print(formes)
    
    if fin == False:
          
        # boucler jusqu'à trouver une correspondance, tout en réduisant le filtre si nécessaire
        while trouve == False and FiltreForme > 3:
            formes = [re.escape(formes_input[x-k][4][:FiltreForme]) for k in range(p)]
            #print("Test avec filtre : ", FiltreForme, formes)

            for line in lines:
                    components = line.split('|')
                    
                    for i in range(len(components) - p):
                        formes_successives = []
                        doublon_precedent = ""
                        
                        if all(forme in components[i - j] for j, forme in enumerate(formes)):
                            #print(line)
                            formes_successives.append(components[i + 1].split('&'))
                            #print(formes_successives)
                            trouve = True
                            
                            for k in range(len(formes_successives)):
                                
                                for i in range(len(formes_successives[k])):
                                    formes_successives[k][i] = formes_successives[k][i].replace(' ', '')
                            #print(formes_successives)

                            # Vérifier et mettre à jour formes_principales_probables
                            for ligne_forme in formes_successives:
                                
                                for forme in ligne_forme:
                                    
                                    doublon = forme[:3]
                                    forme_existe = False
                                    
                                    if doublon != doublon_precedent:
                                        
                                        for i, row in enumerate(formes_principales_probables):
                                            
                                            if row[0][:3] == forme[:3]:
                                                formes_principales_probables[i][p] += 1
                                                forme_existe = True
                                                break
                                            
                                        if not forme_existe:
                                            nouvelle_ligne = [forme[:3]] + [0] * (x + 1)
                                            formes_principales_probables.append(nouvelle_ligne)
                                            formes_principales_probables[-1][p] += 1
                                    doublon_precedent = doublon

                            # Vérifier et mettre à jour formes_precises_probables
                            if FiltreForme == 21:
                                fin = True
                                for ligne_forme in formes_successives:
                                    
                                    for forme in ligne_forme:
                                        forme_existe = False
                                        for i, row in enumerate(formes_precises_probables):
                                            
                                            if row[0] == forme:
                                                formes_precises_probables[i][p] += 1
                                                forme_existe = True
                                                break
                                            
                                        if not forme_existe:
                                            nouvelle_ligne = [forme] + [0] * (x + 1)
                                            formes_precises_probables.append(nouvelle_ligne)
                                            formes_precises_probables[-1][p] += 1
            FiltreForme = FiltreForme - 1
sommes_colonne = []

# Calcul de la moyenne des formes_principales_probables
# Calcul de la moyenne par colonne
for i, col in enumerate(zip(*formes_principales_probables)):
    # Si l'index de la colonne est supérieur ou égal à 1
    if i >= 1:
        # Somme des valeurs de la colonne et ajoute à la liste sum_cols
        sommes_colonne.append(sum(col))

# Initialisation de la liste pour stocker les moyennes de chaque colonne
moyennes_colonnes = []

# Itération sur chaque ligne de formes_precises_probables
for ligne in formes_principales_probables:
    # Initialisation de la nouvelle ligne pour stocker les moyennes
    nouvelle_ligne = [ligne[0]]  # on garde la forme telle quelle
    # Itération sur chaque colonne de la ligne, à partir de la deuxième colonne
    for i in range(1, len(ligne)):
        if sommes_colonne[i - 1] == 0:
            moyenne = 0
        else:
            # Calcul de la moyenne de la colonne
            moyenne = ligne[i] / sommes_colonne[i - 1]  # soustraction de 1 car sum_cols commence à l'index 0
            # Ajout de la moyenne à la nouvelle ligne
            nouvelle_ligne.append(moyenne)
    # Ajout de la nouvelle ligne aux moyennes
    moyennes_colonnes.append(nouvelle_ligne)

# Initialisation de la liste pour stocker les moyennes de chaque ligne
moyennes_ligne = []

# Itération sur chaque ligne de 'moyennes'
for ligne in moyennes_colonnes:
    # Calcul de la moyenne de chaque ligne (en excluant la première colonne qui contient la forme)
    moyenne_ligne = sum(ligne[1:]) / (len(ligne) - 1) # soustraction de 1 car on exclut la première colonne
    # Ajout de la moyenne de la ligne à la deuxième colonne
    ligne.insert(1, moyenne_ligne*100)
    # Ajout de la ligne modifiée à 'moyennes_ligne'
    moyennes_ligne.append(ligne)

# Tri de 'moyennes_ligne' par ordre décroissant en fonction de la deuxième colonne
moyennes_ligne = sorted(moyennes_ligne, key=lambda x: x[1], reverse=True)

best_forme_principale = moyennes_ligne[0][0]
best_score_principal = moyennes_ligne[0][1]

# Calcul de la moyenne des formes_precises_probables
sommes_colonne = []

formes_precises_probables = [ligne for ligne in formes_precises_probables if best_forme_principale in ligne[0][:3]]

# Calcul de la moyenne par colonne
for i, col in enumerate(zip(*formes_precises_probables)):
    # Si l'index de la colonne est supérieur ou égal à 1
    if i >= 1:
        # Somme des valeurs de la colonne et ajoute à la liste sum_cols
        sommes_colonne.append(sum(col))

# Initialisation de la liste pour stocker les moyennes de chaque colonne
moyennes_colonnes = []

# Itération sur chaque ligne de formes_precises_probables
for ligne in formes_precises_probables:
    # Initialisation de la nouvelle ligne pour stocker les moyennes
    nouvelle_ligne = [ligne[0]]  # on garde la forme telle quelle
    # Itération sur chaque colonne de la ligne, à partir de la deuxième colonne
    for i in range(1, len(ligne)):
        if sommes_colonne[i - 1] == 0:
            moyenne = 0
        else:
            # Calcul de la moyenne de la colonne
            moyenne = ligne[i] / sommes_colonne[i - 1] # soustraction de 1 car sum_cols commence à l'index 0
            # Ajout de la moyenne à la nouvelle ligne
            nouvelle_ligne.append(moyenne)
    # Ajout de la nouvelle ligne aux moyennes
    moyennes_colonnes.append(nouvelle_ligne)

# Initialisation de la liste pour stocker les moyennes de chaque ligne
moyennes_ligne = []

# Itération sur chaque ligne de 'moyennes'
for ligne in moyennes_colonnes:
    # Calcul de la moyenne de chaque ligne (en excluant la première colonne qui contient la forme)
    moyenne_ligne = sum(ligne[1:]) / (len(ligne) - 1)  # soustraction de 1 car on exclut la première colonne
    # Ajout de la moyenne de la ligne à la deuxième colonne
    ligne.insert(1, moyenne_ligne*100)
    # Ajout de la ligne modifiée à 'moyennes_ligne'
    moyennes_ligne.append(ligne)

# Tri de 'moyennes_ligne' par ordre décroissant en fonction de la deuxième colonne
moyennes_ligne = sorted(moyennes_ligne, key=lambda x: x[1], reverse=True)

best_forme_precise = moyennes_ligne[0][0]
best_score_precis = moyennes_ligne[0][1]

nouvelle_ligne = ['[...]'] + [0] * (7)
formes_input.append(nouvelle_ligne)
formes_input[-1][1] = best_forme_precise
formes_input[-1][2] = best_forme_principale
formes_input[-1][3] = best_score_principal
formes_input[-1][4] = best_forme_precise
formes_input[-1][5] = best_score_precis
formes_input[-1][6] = 'XX'
formes_input[-1][7] = 0

with open('result.txt', "w", encoding='utf-8', newline="") as fichier:
        writer = csv.writer(fichier, delimiter='|')
        writer.writerows(formes_input)
        
for line in formes_input:
    print(line)
    
print("Prédiction des formes terminée")

Début de la prédiction...
['je', 'PRO:per:suj:e1s', 'PRO', 100.0, 'PRO:per:suj:e1s', 100.0, 'CV', 5556]
['pense', 'VER:P0:A1:sub:pre:3s & VER:P0:A1:sub:pre:1s & VER:P0:A1:ind:pre:3s & VER:P0:A1:ind:pre:1s & VER:P0:A1:imp:2s & VER:P0:A1-N1a:sub:pre:3s & VER:P0:A1-N1a:sub:pre:1s & VER:P0:A1-N1a:ind:pre:3s & VER:P0:A1-N1a:ind:pre:1s & VER:P0:A1-N1a:imp:2s & VER:P0:N1a:sub:pre:3s & VER:P0:N1a:sub:pre:1s & VER:P0:N1a:ind:pre:3s & VER:P0:N1a:ind:pre:1s & VER:P0:N1a:imp:2s & VER:P0:T15:sub:pre:3s & VER:P0:T15:sub:pre:1s & VER:P0:T15:ind:pre:3s & VER:P0:T15:ind:pre:1s & VER:P0:T15:imp:2s & VER:P0:T14:sub:pre:3s & VER:P0:T14:sub:pre:1s & VER:P0:T14:ind:pre:3s & VER:P0:T14:ind:pre:1s & VER:P0:T14:imp:2s & VER:P0:T13-P30:sub:pre:3s & VER:P0:T13-P30:sub:pre:1s & VER:P0:T13-P30:ind:pre:3s & VER:P0:T13-P30:ind:pre:1s & VER:P0:T13-P30:imp:2s', 'VER', 35.54981203007519, 'VER:P0:T14:sub:pre:3s', 0.7142857142857143, 'CV', 217]
['donc', 'CON:coo:csq', 'CON', 100.0, 'CON:coo:csq', 100.0, 'CV', 5556]
['je'

#**D - Affichage du mot suivant probable**



In [38]:
#Ajouter l'indice de créativité

import os
import pandas as pd
import csv
import random
  
indice_creativite = 100

phrase = ""

mots_input= []

df = pd.read_csv('result.txt', sep="|", header=None, encoding='utf-8')

# Itération à travers les lignes
for index, row in df.iterrows():
    mot = row[0]
    id_ = row[1]
    idp = row[2]
    idpp = row[3]
    ide = row[4]
    idep = row[5]
    cv = row[6]
    freq = row[7]

    mots_input.append([mot, id_,  idp , idpp, ide , idep , cv, freq])

forme_principale = mots_input[-1][2]
pourcentage_forme_principale = mots_input[-1][3]
forme_probable = mots_input[-1][4]
pourcentage_forme_probable = mots_input[-1][5]
CV = ""

if mots_input[-2][6][-1] == "V":
  CV = "C"
else:
  CV = "V"

#print(forme)

# lire le fichier
with open('/content/drive/MyDrive/VaCarMe/Dictionnaire.txt', 'r', encoding="utf-8") as file2:
    reader = csv.reader(file2, delimiter='|')

    # initialiser une liste pour stocker les lignes qui correspondent
    mots_probables = []

    # parcourir chaque ligne du Dictionnaire
    for row in reader:
        if forme_probable in row[1]:
          # vérifier si le mot probable doit commencer par une consonne
            if CV == "C":
                if row[2][1] == "C":
                    mots_probables.append(row)
            else:
                mots_probables.append(row)

if len(mots_probables) == 0:
  #print("Erreur !")

  with open('/content/drive/MyDrive/VaCarMe/Dictionnaire.txt', 'r', encoding="utf-8") as file2:
      reader = csv.reader(file2, delimiter='|')

      # initialiser une liste pour stocker les lignes qui correspondent
      mots_probables = []

      # parcourir chaque ligne du Dictionnaire
      for row in reader:
          if forme_probable in row[1]:
              mots_probables.append(row)

mots_probables = sorted(mots_probables, key=lambda row: row[6], reverse=True)
    
x = random.randint(1, 10)

if x > len(mots_probables):
    x = len(mots_probables)
    
mot_choisi = mots_probables[x - 1][0]
formes_probables = mots_probables[x - 1][1]
cv_choisi = mots_probables[x - 1][2]
frequence = mots_probables[x - 1][6]

# vérifier si le mot choisit et le mot précédent sont compatible
if cv_choisi[-2][0][-1] =="V" and CV == "C":
    if mots_input[-2][0] == "ce":
        mots_input[-2][0] = mots_input[-2][0].replace("ce", "c'")
    if mots_input[-2][0] == "je":
        mots_input[-2][0] = mots_input[-2][0].replace("je", "j'")
    if mots_input[-2][0] == "ne":
        mots_input[-2][0] = mots_input[-2][0].replace("ne", "n'")
    if mots_input[-2][0] == "se":
        mots_input[-2][0] = mots_input[-2][0].replace("se", "s'")
    if mots_input[-2][0] == "te":
        mots_input[-2][0] = mots_input[-2][0].replace("te", "t'")
    if mots_input[-2][0] == "que":
        mots_input[-2][0] = mots_input[-2][0].replace("que", "qu'")
    if mots_input[-2][0] == "me":
        mots_input[-2][0] = mots_input[-2][0].replace("me", "m'")
    
# supprimer la dernière ligne de la liste pour la remplacer par le mot choisi
mots_input.pop()

nouvelle_ligne = [mot_choisi] + [0] * (7)
mots_input.append(nouvelle_ligne)
mots_input[-1][1] = formes_probables
mots_input[-1][2] = forme_principale
mots_input[-1][3] = pourcentage_forme_principale
mots_input[-1][4] = forme_probable
mots_input[-1][5] = pourcentage_forme_probable
mots_input[-1][6] = cv_choisi
mots_input[-1][7] = frequence

with open('result.txt', "w", encoding='utf-8', newline="") as fichier:
        writer = csv.writer(fichier, delimiter='|')
        writer.writerows(mots_input)

for line in mots_input:
    #print(line)
    phrase = " ".join(sublist[0] for sublist in mots_input)
    
phrase = phrase.replace(" , ", ",")
phrase = phrase.replace("' ", "'")

print(phrase)

Erreur !
je pense donc j'ai
